# Chicago Car Crash Analysis

-Past 3 years (2020-2022)  
-Total Injuries (not 0)  
-Only 4 wheelers
-Target is PRIM_CONTRIBUTORY_CAUSE

-business problem: determining primary contributory causes of crashes from 4 wheel passenger vehicles that resulted in an injury in the past 3 years
-yes eda
-yes, lots of preprocessing required One hot, ordinal, 
-baseline model - decision tree, logistic regression, 
    metrics - precision - we want to focus on correctness of determining these causes


-multiple binary classification problems if we want to use logistic regression
-two-class classification problems

-graph prim_contributory_cause against vehicle_type
-or against year
-prim_contributory_cause vs injuries_total



Steps for now:  
1. Filter for   
    -Past 3 years (2020-2022)  
    -Total Injuries (not 0)  
    -
2. Drop all columns don't need  
3. divide data cleaning  


GRAPHS:

-primary contributory cause against year, time, age


In [9]:
#imports 
import pandas as pd
import numpy as np

In [10]:
#read data in
df_crashes = pd.read_csv("data/Traffic_Crashes_-_Crashes_20231109.csv")
df_people = pd.read_csv("data/Traffic_Crashes_-_People_20231109.csv")
df_vehicles = pd.read_csv("data/Traffic_Crashes_-_Vehicles_20231109.csv")

In [11]:
df_crashes['CRASH_DATE'] = pd.to_datetime(df_crashes['CRASH_DATE'])
df_crashes['YEAR'] = df_crashes['CRASH_DATE'].dt.year

In [17]:
df_crashes['YEAR'].value_counts()
df_crashes.loc[df_crashes['YEAR']==2023]['MOST_SEVERE_INJURY'].value_counts()

NO INDICATION OF INJURY     80408
NONINCAPACITATING INJURY     8223
REPORTED, NOT EVIDENT        4310
INCAPACITATING INJURY        1676
FATAL                         121
Name: MOST_SEVERE_INJURY, dtype: int64

**CRASHES dataframe filtering:**

In [4]:
#drop useless columns in crashes df
crash_columns_to_drop = ["RD_NO", "CRASH_DATE_EST_I","REPORT_TYPE", 'DATE_POLICE_NOTIFIED', 'SEC_CONTRIBUTORY_CAUSE', 'STREET_NO', 'STREET_DIRECTION', 'STREET_NAME', 'BEAT_OF_OCCURRENCE', "PHOTOS_TAKEN_I","STATEMENTS_TAKEN_I",'DOORING_I',"WORK_ZONE_I",'WORK_ZONE_TYPE', "WORKERS_PRESENT_I",'LATITUDE', 'LONGITUDE', 'LOCATION']

In [5]:
df_crashes.drop(crash_columns_to_drop, axis=1, inplace=True)

In [6]:
#sort by dates past 2020 and injuries only
df_crashes = df_crashes[df_crashes["CRASH_DATE"] > '01/01/2020']
df_crashes = df_crashes[df_crashes["INJURIES_TOTAL"] > 0]

In [7]:
df_crashes.head()
#df_crashes['PRIM']

,CRASH_RECORD_ID,CRASH_DATE,POSTED_SPEED_LIMIT,TRAFFIC_CONTROL_DEVICE,DEVICE_CONDITION,WEATHER_CONDITION,LIGHTING_CONDITION,FIRST_CRASH_TYPE,TRAFFICWAY_TYPE,LANE_CNT,...,INJURIES_TOTAL,INJURIES_FATAL,INJURIES_INCAPACITATING,INJURIES_NON_INCAPACITATING,INJURIES_REPORTED_NOT_EVIDENT,INJURIES_NO_INDICATION,INJURIES_UNKNOWN,CRASH_HOUR,CRASH_DAY_OF_WEEK,CRASH_MONTH
0,6c1659069e9c6285a650e70d6f9b574ed5f64c12888479...,08/18/2023 12:50:00 PM,15,OTHER,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,REAR END,OTHER,NaN,...,1.0,0.0,0.0,1.0,0.0,1.0,0.0,12,6,8
2,61fcb8c1eb522a6469b460e2134df3d15f82e81fd93e9c...,08/18/2023 05:58:00 PM,30,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,PEDALCYCLIST,NOT DIVIDED,NaN,...,1.0,0.0,0.0,1.0,0.0,1.0,0.0,17,6,8
3,004cd14d0303a9163aad69a2d7f341b7da2a8572b2ab33...,11/26/2019 08:38:00 AM,25,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,PEDESTRIAN,ONE-WAY,NaN,...,1.0,1.0,0.0,0.0,0.0,1.0,0.0,8,3,11
21,3facd2ed99a27da2f5572605b9434167a6fa7777243e04...,09/27/2019 06:00:00 PM,30,NO CONTROLS,NO CONTROLS,RAIN,DUSK,PEDESTRIAN,FOUR WAY,NaN,...,1.0,0.0,0.0,1.0,0.0,1.0,0.0,18,6,9
23,1ee2180a89cc02c0b756f95b5b2755bb5cc9d93450f5ca...,08/09/2023 07:55:00 PM,30,TRAFFIC SIGNAL,FUNCTIONING IMPROPERLY,CLEAR,DAYLIGHT,ANGLE,FOUR WAY,NaN,...,5.0,0.0,0.0,5.0,0.0,0.0,0.0,19,4,8


In [6]:
df_crashes['PRIM_CONTRIBUTORY_CAUSE'].value_counts()

UNABLE TO DETERMINE                                                                 302153
FAILING TO YIELD RIGHT-OF-WAY                                                        85315
FOLLOWING TOO CLOSELY                                                                76341
NOT APPLICABLE                                                                       41267
IMPROPER OVERTAKING/PASSING                                                          38280
FAILING TO REDUCE SPEED TO AVOID CRASH                                               32933
IMPROPER BACKING                                                                     30962
IMPROPER LANE USAGE                                                                  28112
DRIVING SKILLS/KNOWLEDGE/EXPERIENCE                                                  25874
IMPROPER TURNING/NO SIGNAL                                                           25859
DISREGARDING TRAFFIC SIGNALS                                                         15118

**VEHICLES dataframe filtering:**

In [10]:
#drop useless columns in vehicles df
vehicles_columns = ['CRASH_RECORD_ID', 'NUM_PASSENGERS', 'CMRC_VEH_I', 'MAKE', 'MODEL', 'LIC_PLATE_STATE', 'VEHICLE_YEAR', 'VEHICLE_DEFECT','VEHICLE_TYPE', 'VEHICLE_USE', 'MANEUVER', 'EXCEED_SPEED_LIMIT_I', 'FIRST_CONTACT_POINT', 'TOTAL_VEHICLE_LENGTH', 'VEHICLE_CONFIG']

In [11]:
df_vehicles=df_vehicles[vehicles_columns]
df_vehicles=df_vehicles[(df_vehicles['VEHICLE_TYPE'] == "PASSENGER") | (df_vehicles["VEHICLE_TYPE"] == "SPORT UTILITY VEHICLE (SUV)") | (df_vehicles["VEHICLE_TYPE"] == "VAN/MINI-VAN")]

In [12]:
df_vehicles.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1262351 entries, 0 to 1588936
Data columns (total 15 columns):
 #   Column                Non-Null Count    Dtype  
---  ------                --------------    -----  
 0   CRASH_RECORD_ID       1262351 non-null  object 
 1   NUM_PASSENGERS        211590 non-null   float64
 2   CMRC_VEH_I            4977 non-null     object 
 3   MAKE                  1262346 non-null  object 
 4   MODEL                 1262220 non-null  object 
 5   LIC_PLATE_STATE       1189166 non-null  object 
 6   VEHICLE_YEAR          1147035 non-null  float64
 7   VEHICLE_DEFECT        1262351 non-null  object 
 8   VEHICLE_TYPE          1262351 non-null  object 
 9   VEHICLE_USE           1262351 non-null  object 
 10  MANEUVER              1262351 non-null  object 
 11  EXCEED_SPEED_LIMIT_I  1997 non-null     object 
 12  FIRST_CONTACT_POINT   1260202 non-null  object 
 13  TOTAL_VEHICLE_LENGTH  27 non-null       float64
 14  VEHICLE_CONFIG        301 non-null

**PEOPLE dataframe filtering:**

In [13]:
df_people.columns

Index(['PERSON_ID', 'PERSON_TYPE', 'CRASH_RECORD_ID', 'RD_NO', 'VEHICLE_ID',
       'CRASH_DATE', 'SEAT_NO', 'CITY', 'STATE', 'ZIPCODE', 'SEX', 'AGE',
       'DRIVERS_LICENSE_STATE', 'DRIVERS_LICENSE_CLASS', 'SAFETY_EQUIPMENT',
       'AIRBAG_DEPLOYED', 'EJECTION', 'INJURY_CLASSIFICATION', 'HOSPITAL',
       'EMS_AGENCY', 'EMS_RUN_NO', 'DRIVER_ACTION', 'DRIVER_VISION',
       'PHYSICAL_CONDITION', 'PEDPEDAL_ACTION', 'PEDPEDAL_VISIBILITY',
       'PEDPEDAL_LOCATION', 'BAC_RESULT', 'BAC_RESULT VALUE',
       'CELL_PHONE_USE'],
      dtype='object')

In [14]:
people_columns = ['CRASH_RECORD_ID', 'SEX', 'AGE', 'SAFETY_EQUIPMENT','AIRBAG_DEPLOYED', 'INJURY_CLASSIFICATION', 'DRIVER_ACTION', 'DRIVER_VISION', 'PHYSICAL_CONDITION','PEDPEDAL_ACTION', 'PEDPEDAL_VISIBILITY','PEDPEDAL_LOCATION', 'BAC_RESULT', 'BAC_RESULT VALUE','CELL_PHONE_USE']
#sex, age, safety_equipment, airbag, injury classfication, driver_action, vision, physical_condi, ped location, bac, cell use

In [15]:
df_people = df_people[people_columns]

In [16]:
df_people.head()

,CRASH_RECORD_ID,SEX,AGE,SAFETY_EQUIPMENT,AIRBAG_DEPLOYED,INJURY_CLASSIFICATION,DRIVER_ACTION,DRIVER_VISION,PHYSICAL_CONDITION,PEDPEDAL_ACTION,PEDPEDAL_VISIBILITY,PEDPEDAL_LOCATION,BAC_RESULT,BAC_RESULT VALUE,CELL_PHONE_USE
0,81dc0de2ed92aa62baccab641fa377be7feb1cc47e6554...,M,25.0,NONE PRESENT,DEPLOYMENT UNKNOWN,NO INDICATION OF INJURY,UNKNOWN,UNKNOWN,UNKNOWN,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
1,af84fb5c8d996fcd3aefd36593c3a02e6e7509eeb27568...,M,37.0,SAFETY BELT USED,DID NOT DEPLOY,NO INDICATION OF INJURY,NONE,NOT OBSCURED,NORMAL,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
2,71162af7bf22799b776547132ebf134b5b438dcf3dac6b...,X,NaN,USAGE UNKNOWN,DEPLOYMENT UNKNOWN,NO INDICATION OF INJURY,IMPROPER BACKING,UNKNOWN,UNKNOWN,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
3,c21c476e2ccc41af550b5d858d22aaac4ffc88745a1700...,X,NaN,USAGE UNKNOWN,DEPLOYMENT UNKNOWN,NO INDICATION OF INJURY,UNKNOWN,UNKNOWN,UNKNOWN,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
4,eb390a4c8e114c69488f5fb8a097fe629f5a92fd528cf4...,X,NaN,USAGE UNKNOWN,DEPLOYMENT UNKNOWN,NO INDICATION OF INJURY,UNKNOWN,UNKNOWN,UNKNOWN,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN


In [17]:
df_joined = pd.merge(df_crashes, pd.merge(df_vehicles, df_people, on="CRASH_RECORD_ID"), on="CRASH_RECORD_ID")

In [18]:
#dropping extra columns:
df_joined.drop(["LANE_CNT","INTERSECTION_RELATED_I","NOT_RIGHT_OF_WAY_I","HIT_AND_RUN_I","INJURIES_FATAL","INJURIES_INCAPACITATING","INJURIES_NON_INCAPACITATING","INJURIES_REPORTED_NOT_EVIDENT","INJURIES_NO_INDICATION","INJURIES_UNKNOWN","NUM_PASSENGERS","CMRC_VEH_I","MAKE","MODEL","VEHICLE_YEAR","VEHICLE_DEFECT","VEHICLE_USE","EXCEED_SPEED_LIMIT_I", "TOTAL_VEHICLE_LENGTH","VEHICLE_CONFIG","PEDPEDAL_ACTION","PEDPEDAL_VISIBILITY","PEDPEDAL_LOCATION","BAC_RESULT VALUE","CELL_PHONE_USE"],axis=1, inplace=True)

In [19]:
df_joined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 533246 entries, 0 to 533245
Data columns (total 34 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   CRASH_RECORD_ID          533246 non-null  object 
 1   CRASH_DATE               533246 non-null  object 
 2   POSTED_SPEED_LIMIT       533246 non-null  int64  
 3   TRAFFIC_CONTROL_DEVICE   533246 non-null  object 
 4   DEVICE_CONDITION         533246 non-null  object 
 5   WEATHER_CONDITION        533246 non-null  object 
 6   LIGHTING_CONDITION       533246 non-null  object 
 7   FIRST_CRASH_TYPE         533246 non-null  object 
 8   TRAFFICWAY_TYPE          533246 non-null  object 
 9   ALIGNMENT                533246 non-null  object 
 10  ROADWAY_SURFACE_COND     533246 non-null  object 
 11  ROAD_DEFECT              533246 non-null  object 
 12  CRASH_TYPE               533246 non-null  object 
 13  DAMAGE                   533246 non-null  object 
 14  PRIM

In [30]:
df_joined["INJURIES_TOTAL"].value_counts()

1.0     327661
2.0     117144
3.0      47756
4.0      21981
5.0       9216
6.0       4464
7.0       2270
8.0        941
9.0        640
10.0       395
11.0       160
17.0       136
12.0       118
15.0       108
16.0        85
21.0        67
13.0        53
19.0        29
14.0        22
Name: INJURIES_TOTAL, dtype: int64

In [23]:
df_joined.isna().sum()

CRASH_RECORD_ID                 0
CRASH_DATE                      0
POSTED_SPEED_LIMIT              0
TRAFFIC_CONTROL_DEVICE          0
DEVICE_CONDITION                0
WEATHER_CONDITION               0
LIGHTING_CONDITION              0
FIRST_CRASH_TYPE                0
TRAFFICWAY_TYPE                 0
ALIGNMENT                       0
ROADWAY_SURFACE_COND            0
ROAD_DEFECT                     0
CRASH_TYPE                      0
DAMAGE                          0
PRIM_CONTRIBUTORY_CAUSE         0
NUM_UNITS                       0
MOST_SEVERE_INJURY              0
INJURIES_TOTAL                  0
CRASH_HOUR                      0
CRASH_DAY_OF_WEEK               0
CRASH_MONTH                     0
LIC_PLATE_STATE             18571
VEHICLE_TYPE                    0
MANEUVER                        0
FIRST_CONTACT_POINT           861
SEX                         11066
AGE                         89876
SAFETY_EQUIPMENT             3099
AIRBAG_DEPLOYED             22388
INJURY_CLASSIF

In [25]:
df_joined["VEHICLE_TYPE"].value_counts()

PASSENGER                      416119
SPORT UTILITY VEHICLE (SUV)     85939
VAN/MINI-VAN                    31188
Name: VEHICLE_TYPE, dtype: int64

In [28]:
df_joined.head()

,CRASH_RECORD_ID,CRASH_DATE,POSTED_SPEED_LIMIT,TRAFFIC_CONTROL_DEVICE,DEVICE_CONDITION,WEATHER_CONDITION,LIGHTING_CONDITION,FIRST_CRASH_TYPE,TRAFFICWAY_TYPE,ALIGNMENT,...,FIRST_CONTACT_POINT,SEX,AGE,SAFETY_EQUIPMENT,AIRBAG_DEPLOYED,INJURY_CLASSIFICATION,DRIVER_ACTION,DRIVER_VISION,PHYSICAL_CONDITION,BAC_RESULT
0,6c1659069e9c6285a650e70d6f9b574ed5f64c12888479...,08/18/2023 12:50:00 PM,15,OTHER,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,REAR END,OTHER,STRAIGHT AND LEVEL,...,FRONT,M,45.0,SAFETY BELT USED,NOT APPLICABLE,NO INDICATION OF INJURY,FAILED TO YIELD,NOT OBSCURED,NORMAL,TEST NOT OFFERED
1,6c1659069e9c6285a650e70d6f9b574ed5f64c12888479...,08/18/2023 12:50:00 PM,15,OTHER,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,REAR END,OTHER,STRAIGHT AND LEVEL,...,FRONT,M,69.0,HELMET NOT USED,NOT APPLICABLE,NONINCAPACITATING INJURY,NONE,NOT OBSCURED,NORMAL,TEST NOT OFFERED
2,1ee2180a89cc02c0b756f95b5b2755bb5cc9d93450f5ca...,08/09/2023 07:55:00 PM,30,TRAFFIC SIGNAL,FUNCTIONING IMPROPERLY,CLEAR,DAYLIGHT,ANGLE,FOUR WAY,STRAIGHT AND LEVEL,...,FRONT,F,19.0,USAGE UNKNOWN,DID NOT DEPLOY,NONINCAPACITATING INJURY,NaN,NaN,NaN,NaN
3,1ee2180a89cc02c0b756f95b5b2755bb5cc9d93450f5ca...,08/09/2023 07:55:00 PM,30,TRAFFIC SIGNAL,FUNCTIONING IMPROPERLY,CLEAR,DAYLIGHT,ANGLE,FOUR WAY,STRAIGHT AND LEVEL,...,FRONT,F,33.0,USAGE UNKNOWN,"DEPLOYED, FRONT",NONINCAPACITATING INJURY,UNKNOWN,UNKNOWN,NORMAL,TEST NOT OFFERED
4,1ee2180a89cc02c0b756f95b5b2755bb5cc9d93450f5ca...,08/09/2023 07:55:00 PM,30,TRAFFIC SIGNAL,FUNCTIONING IMPROPERLY,CLEAR,DAYLIGHT,ANGLE,FOUR WAY,STRAIGHT AND LEVEL,...,FRONT,M,38.0,USAGE UNKNOWN,DID NOT DEPLOY,NONINCAPACITATING INJURY,UNKNOWN,NOT OBSCURED,NORMAL,TEST NOT OFFERED


In [27]:
df_joined['CRASH_DATE'].sort_values(ascending=False)

415768    12/31/2022 12:08:00 PM
415767    12/31/2022 12:08:00 PM
415766    12/31/2022 12:08:00 PM
89285     12/31/2022 12:00:00 PM
89288     12/31/2022 12:00:00 PM
                   ...          
267612    01/01/2020 01:24:00 AM
267611    01/01/2020 01:24:00 AM
267607    01/01/2020 01:24:00 AM
519306    01/01/2020 01:10:00 PM
519307    01/01/2020 01:10:00 PM
Name: CRASH_DATE, Length: 533246, dtype: object

Grouping prim contributory causes 
dropped columns:
-unable to determine
-not applicable

Columns We choose:
1. failing to yield right of way
2. following too closely
3. improper overtaking/passing
4. failing to reduce speed 
5. improper backing
6. improper lane usage
7. driving skills/knowledge/experience
8. improper turning/no signal
9. disregarding traffic signals + disregarding stop sign + DISREGARDING OTHER TRAFFIC SIGNS
10. reckless driving (operating vehicle in reckless manner + UNDER THE INFLUENCE OF ALCOHOL/DRUGS + EXCEEDING SAFE SPEED FOR CONDITIONS + 
11. non human error (weather + equipment-vehicle condition + EVASIVE ACTION DUE TO ANIMAL, OBJECT, NONMOTORIST + animal + VISION OBSCURED



In [26]:
df_crashes.isna().sum()

CRASH_RECORD_ID                       0
RD_NO                              4555
CRASH_DATE_EST_I                 720341
CRASH_DATE                            0
POSTED_SPEED_LIMIT                    0
TRAFFIC_CONTROL_DEVICE                0
DEVICE_CONDITION                      0
WEATHER_CONDITION                     0
LIGHTING_CONDITION                    0
FIRST_CRASH_TYPE                      0
TRAFFICWAY_TYPE                       0
LANE_CNT                         579841
ALIGNMENT                             0
ROADWAY_SURFACE_COND                  0
ROAD_DEFECT                           0
REPORT_TYPE                       22589
CRASH_TYPE                            0
INTERSECTION_RELATED_I           600409
NOT_RIGHT_OF_WAY_I               742793
HIT_AND_RUN_I                    535597
DAMAGE                                0
DATE_POLICE_NOTIFIED                  0
PRIM_CONTRIBUTORY_CAUSE               0
SEC_CONTRIBUTORY_CAUSE                0
STREET_NO                             0


In [37]:
df_people.isna().sum()

PERSON_ID                      0
PERSON_TYPE                    0
CRASH_RECORD_ID                0
RD_NO                      10080
VEHICLE_ID                 34155
CRASH_DATE                     0
SEAT_NO                  1364061
CITY                      464292
STATE                     446368
ZIPCODE                   568500
SEX                        27963
AGE                       498987
DRIVERS_LICENSE_STATE     707332
DRIVERS_LICENSE_CLASS     868869
SAFETY_EQUIPMENT            4777
AIRBAG_DEPLOYED            32863
EJECTION                   21013
INJURY_CLASSIFICATION        711
HOSPITAL                 1423381
EMS_AGENCY               1532314
EMS_RUN_NO               1680618
DRIVER_ACTION             349088
DRIVER_VISION             349596
PHYSICAL_CONDITION        348158
PEDPEDAL_ACTION          1677585
PEDPEDAL_VISIBILITY      1677644
PEDPEDAL_LOCATION        1677586
BAC_RESULT                348109
BAC_RESULT VALUE         1708138
CELL_PHONE_USE           1708920
dtype: int

In [70]:
df_people['PEDPEDAL_LOCATION'].value_counts()

IN ROADWAY         14454
IN CROSSWALK       10722
UNKNOWN/NA          2727
NOT IN ROADWAY      1437
BIKEWAY             1431
BIKE LANE           1076
DRIVEWAY ACCESS      464
SHOULDER             181
Name: PEDPEDAL_LOCATION, dtype: int64

In [69]:
df_vehicles["VEHICLE_TYPE"].value_counts()

PASSENGER                                 976728
SPORT UTILITY VEHICLE (SUV)               211332
UNKNOWN/NA                                145049
VAN/MINI-VAN                               74291
PICKUP                                     49599
TRUCK - SINGLE UNIT                        29351
OTHER                                      18631
BUS OVER 15 PASS.                          16464
TRACTOR W/ SEMI-TRAILER                    14796
BUS UP TO 15 PASS.                          4175
MOTORCYCLE (OVER 150CC)                     3764
SINGLE UNIT TRUCK WITH TRAILER              2506
OTHER VEHICLE WITH TRAILER                  2072
TRACTOR W/O SEMI-TRAILER                    1959
AUTOCYCLE                                    665
MOPED OR MOTORIZED BICYCLE                   502
MOTOR DRIVEN CYCLE                           326
ALL-TERRAIN VEHICLE (ATV)                    177
FARM EQUIPMENT                                81
3-WHEELED MOTORCYCLE (2 REAR WHEELS)          56
RECREATIONAL OFF-HIG

In [33]:
crash_columns_to_drop = ["PHOTOS_TAKEN_I","STATEMENTS_TAKEN_I","DOORING_I","WORK_ZONE_I","WORK_ZONE_TYPE","WORKERS_PRESENT_I"]

df_crashes.drop(crash_columns_to_drop, axis=1, inplace=True)

In [71]:
df_vehicles["TOTAL_VEHICLE_LENGTH"].value_counts()

53.0     275
65.0     267
40.0     210
75.0     198
60.0     153
        ... 
95.0       1
31.0       1
101.0      1
103.0      1
196.0      1
Name: TOTAL_VEHICLE_LENGTH, Length: 125, dtype: int64

In [35]:
df_crash.isna().sum()

CRASH_RECORD_ID                       0
RD_NO                              4555
CRASH_DATE_EST_I                 720341
CRASH_DATE                            0
POSTED_SPEED_LIMIT                    0
TRAFFIC_CONTROL_DEVICE                0
DEVICE_CONDITION                      0
WEATHER_CONDITION                     0
LIGHTING_CONDITION                    0
FIRST_CRASH_TYPE                      0
TRAFFICWAY_TYPE                       0
LANE_CNT                         579841
ALIGNMENT                             0
ROADWAY_SURFACE_COND                  0
ROAD_DEFECT                           0
REPORT_TYPE                       22589
CRASH_TYPE                            0
INTERSECTION_RELATED_I           600409
NOT_RIGHT_OF_WAY_I               742793
HIT_AND_RUN_I                    535597
DAMAGE                                0
DATE_POLICE_NOTIFIED                  0
PRIM_CONTRIBUTORY_CAUSE               0
SEC_CONTRIBUTORY_CAUSE                0
STREET_NO                             0


In [73]:
#df_crash.join(df_people.set_index('CRASH_RECORD_ID'), on='CRASH_RECORD_ID', how="left")

In [29]:
df_people['BAC_RESULT VALUE'].value_counts()

0.00    190
0.18    134
0.17    129
0.21    116
0.14    109
0.20    102
0.16     99
0.19     91
0.15     82
0.22     80
0.23     75
0.11     72
0.13     71
0.12     70
0.24     65
0.26     48
0.25     45
0.10     36
0.27     35
0.09     33
0.28     32
0.08     25
0.29     19
0.07     18
0.03     17
0.30     17
0.33     16
0.04     15
0.05     11
0.32     10
0.35      9
0.06      8
0.02      8
0.31      8
0.38      7
0.36      4
0.34      4
0.39      3
0.44      3
0.01      3
1.00      2
0.60      2
0.88      2
0.79      2
0.45      2
0.80      1
0.98      1
0.95      1
0.99      1
0.85      1
0.58      1
0.47      1
0.37      1
0.67      1
0.41      1
0.40      1
Name: BAC_RESULT VALUE, dtype: int64